<a href="https://colab.research.google.com/github/bowcai/YOLOv3/blob/main/demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Run the following block if in Google Colab.

In [1]:
import os
from subprocess import getoutput
clone_output = getoutput("git clone https://github.com/bowcai/YOLOv3.git")
print(clone_output)
os.chdir('YOLOv3')

Cloning into 'YOLOv3'...


In [2]:
import os
from yolov3.load_data import create_training_instances, BatchGenerator
from yolov3.load_weight import load_keras_model, save_keras_model, load_pretrained_weight
from yolov3.model import create_yolov3_model
from yolov3.train import fit_model
from yolov3.evaluate import evaluate

In [4]:

# The path of training and validation set.
train_img_folder = ''
train_annot_folder = ''
train_cache_dir = ''
valid_img_folder = None
valid_annot_folder = None
valid_cache_dir = None

# The labels of object classes.
labels = []

# Batch size.
batch_size = 16

# Warmup batches.
warmup_batches = 3

# If the IoU of predicted bounding box with any ground truth box is higher than the threshold,
# ignore the loss of the predicted box.
ignore_thresh = 0.5

grid_scales = [1, 1, 1]
obj_scale = 5
noobj_scale = 1
xywh_scale = 1
class_scale = 1

# Minimum and maximum input size.
min_input_size = 288
max_input_size = 448

# The pre-defined anchor sizes.
anchors = [55, 69, 75, 234, 133, 240, 136, 129, 142, 363, 203, 290, 228, 184, 285, 359, 341, 260]

# Ratio between network input's size and network output's size, 32 for YOLOv3.
downsample = 32

# The path of pre-trained YOLOv3 weights.
pretrained_weight_path = './weights/yolov3.weights'

# The path of trained Keras model.
keras_model_path = './weights/trained_model.h5'
